# **SAURAN: Data quality checking program v1.5**
This program is designed to perform data quality checks and formatting on weather datasets (csv files) downloaded from the SAURAN website (https://sauran.ac.za/).

For more information on the quality checking of data, please consult the document that came with this program. Please note that this quality checking is performed on mintue or hourly resolution data and **not on daily averaged data**. Also note that larger minute-averaged files may take a while to upload. It is recommended that Google Chrome is used as the web browser when running this tool.

This code was written in Python, and is available below. If you would like to report bugs, problems with the program or require more information, please contact Don Fitzgerald at don@sun.ac.za or the CRSES at crses@sun.ac.za.

**Discliamer: Use this program at your own risk. The CRSES takes no responsibility for incorrect results generated by this tool.**




In [ ]:
#@title <-- click the play button, wait for it to finish and then move onto the next cell. {display-mode: "form"}
#Initiating code imports
!pip install XlsxWriter
import pandas as pd
import sys
import os
print('Program started, please move onto the next cell.')

In [ ]:
#@title Hit play and upload a weather file from the SAURAN website (*.csv). {display-mode: "form"}

# Google Colab import of csv file
import io
from google.colab import files
uploadedFile = files.upload()
uploadedFileName = next(iter(uploadedFile))

try:
  #Extract the stations name
  dataset_stationName = pd.read_csv(io.BytesIO(uploadedFile[uploadedFileName]), sep=',', header=0)
  stationName = dataset_stationName.columns[0]
  
  dataset = pd.read_csv(io.BytesIO(uploadedFile[uploadedFileName]), sep=',', header=1)

except UnicodeDecodeError:
  print('UPLOAD FAILED!')
  print('Please make sure the file format is *.csv and try re-upload by pushing the play button of this cell again.')
  print('Also make sure the file still contains the station name in the first cell.')

else:
  runTest = True
  newPath = os.path.splitext(uploadedFileName)[0]
  newPath =newPath + "_output.xlsx"

  writer = pd.ExcelWriter(newPath, engine = 'xlsxwriter')
  print('Upload successful!')

  print('Please make sure the data is correct before moving on:')
  dataset_v1 = dataset.copy()

  #Removing NaN values at the end of the data
  dataset_firstTwoLines = dataset_v1.iloc[:2]
  concatData = [dataset_firstTwoLines, dataset_v1.iloc[2:].dropna(how = 'all')]
  dataset_v1 = pd.concat(concatData)
  display(dataset_v1)

# #Manual import of CSV file in the same directory:

# pwd = os.getcwd()

# #Enter the file name of the CSV file below:
# path = pwd + "\\SUN Hour.csv"

# rename = os.path.splitext(path)[0]
# rename = os.path.basename(rename)
# newPath = pwd + "\\" + rename + "_output.xlsx"

# dataset = pd.read_csv(path, sep=',', header=1)
# dataset_v1 = dataset.copy()

# writer = pd.ExcelWriter(newPath, engine = 'xlsxwriter')

In [ ]:
#@title Hit play to process the data and download the output file. {display-mode: "form"}

#Calculating the resolution of the data
dataset_resTest = dataset_v1.copy()
resPoint1 = pd.to_datetime(dataset_resTest.iloc[2][0], format='%d/%m/%Y %H:%M:%S')
resPoint2 = pd.to_datetime(dataset_resTest.iloc[3][0], format='%d/%m/%Y %H:%M:%S')

resolution = resPoint2 - resPoint1
resolution = resolution.days*24 + resolution.seconds/3600

#Test which weather station is being analysed
if ('unz' or 'unv' or 'csir' or 'upr' or 'ubg' or 'nust' or 'rvd' or 'van' or 'mrb' or 'grt' or 'ufs' or 'frh' or 'cut' in stationName.lower()) and (resolution == 1 or resolution == (60/3600)):
  dataset_v1.rename(columns = {'TmStamp': 'Time_stamp', 'DNI_Avg': 'DNI', 'DIF_Avg': 'DHI', 'GHI_Avg': 'GHI', 'DNICalc_Avg': 'DNIcalc'}, inplace = True)

if ('sun' in stationName.lower()) and (resolution == 1 or resolution == (60/3600)):
  dataset_v1.rename(columns = {'TmStamp': 'Time_stamp', 'TrackerWM_Avg': 'DNI', 'ShadowWM_Avg': 'DHI', 'SunWM_Avg': 'GHI', 'DNICalc_Avg': 'DNIcalc'}, inplace = True)

if ('salt' or 'sut' in stationName.lower()) and (resolution == 1 or resolution == (60/3600)):
  dataset_v1.rename(columns = {'TmStamp': 'Time_stamp', 'Sol_Direct_Avg': 'DNI', 'Sol_Global2_Avg': 'DHI', 'Sol_Global1_Avg': 'GHI', 'DNICalc_Avg': 'DNIcalc'}, inplace = True)

if ('hlo' in stationName.lower()) and (resolution == 1 or resolution == (60/3600)):
  dataset_v1.rename(columns = {'TmStamp': 'Time_stamp', 'DNI_Avg': 'DNI', 'DIF_Avg': 'DHI', 'GHI1_Avg': 'GHI', 'DNICalc_Avg': 'DNIcalc'}, inplace = True)

if ('pmb' in stationName.lower()) and (resolution == 1 or resolution == (60/3600)):
  dataset_v1.rename(columns = {'TmStamp': 'Time_stamp', 'TrackerWM_Avg': 'DNI', 'ShadowWM_Avg': 'DHI', 'SunWM_Avg': 'GHI', 'DNI_Calc_1_Avg': 'DNIcalc'}, inplace = True)

if 'kzh' in stationName.lower():
  print('Quality checking is not performed on the KZN: KwaZulu-Natal Howard College station as it lacks sufficient checking data (specifically: DNI_calc).')
  print('Please consult the documentation for details on how to perform quality checking on this station yourself.')
  sys.exit()

if 'sta' in stationName.lower():
  print('Quality checking is not performed on the Mangosuthu University of Technology station as it lacks sufficient checking data (specifically: DNI_calc).')
  print('Please consult the documentation for details on how to perform quality checking on this station yourself.')
  sys.exit()

if 'nmu' in stationName.lower():
  print('Quality checking is not performed on the Nelson Mandela University station as it lacks sufficient checking data (specifically: DNI_calc).')
  print('Please consult the documentation for details on how to perform quality checking on this station yourself.')
  sys.exit()

if 'kzw' in stationName.lower():
  print('Quality checking is not performed on the University of KwaZulu-Natal Westville as it lacks sufficient checking data (specifically: DNI_calc).')
  print('Please consult the documentation for details on how to perform quality checking on this station yourself.')
  sys.exit()  

if resolution == 24:  
  print('Sorry, this program does not perform quality and format checking on data with a daily resolution.')
  print('Please use either hourly- or minute-averaged data to perform quality checking.')
  sys.exit()

if resolution != 1 and resolution != (60/3600) and resolution != 24:
  print('There was a problem with the calculation of the resolution!')
  print('Please make sure there are no missing time stamps in the first 3 lines of data and try upload again.')
  sys.exit()

#Drop any unwanted columns
if 'nust' in stationName.lower():
  dataset_v1 = dataset_v1.drop(columns = ['LoggerStationName', 'LoggerSerialNumber'])

#Look for NaN values in the 'Original Data' dataset in row 0 and replace them with 'No unit'
dataset_originalData = dataset_v1.copy()
dataset_originalData.iloc[0] = dataset_originalData.iloc[0].fillna("No unit")

#Assign headers for the 'Original Data' dataset
dataset_originalData.columns = dataset_originalData.columns + " (" + dataset_originalData.iloc[0] + ")"
dataset_originalData = dataset_originalData.iloc[2:] #drops the first 2 rows
dataset_originalData = dataset_originalData.reset_index(drop=True)

#Assign headers for the 'Quality Checking' dataset
dataset_v1 = dataset_v1.iloc[2:] #drops the first 2 rows
dataset_v1 = dataset_v1.reset_index(drop=True)

#Converting to dates and numeric
dataset_v1['Time_stamp'] = pd.to_datetime(dataset_v1['Time_stamp'], format='%d/%m/%Y %H:%M:%S')
colsConvert = dataset_v1.columns.drop('Time_stamp')
dataset_v1[colsConvert] = dataset_v1[colsConvert].apply(pd.to_numeric)

#Setting up a new sheet
dataset_v2 = dataset_v1.copy()
dataset_v2.drop(dataset_v2.columns.difference(['Time_stamp','DNI', 'DHI', 'GHI', 'DNIcalc']), 1, inplace=True)

#Look for missing time stamps
rowNum = len(dataset_v2.axes[0]) - 1

initialDate = dataset_v2.iloc[0][0]
lastDate = dataset_v2.iloc[rowNum][0]

dataPointTotal = (lastDate - initialDate)
dataPointTotal = (dataPointTotal.days*24 + dataPointTotal.seconds/3600)/resolution

missingTimestamps = dataPointTotal - rowNum

#Establishing DNI error
dataset_v2["DNI_error"] = abs((dataset_v2["DNIcalc"] - dataset_v2["DNI"]) / dataset_v2["DNIcalc"])
dataset_v2["DNI_error"].fillna(0, inplace=True)

for i in range(len(dataset_v2.axes[0])):
    hour = dataset_v2.iloc[i]["Time_stamp"].hour

    #Filtering out morning and evening (low-light conditions)
    if hour < 10 or hour > 15:
        dataset_v2.at[i, "DNI_error"] = 0

    #Filtering out daytime low-light conditions below 5 W/m2
    elif dataset_v2.at[i, "DNIcalc"] < 5:
      dataset_v2.at[i, "DNI_error"] = 0
    
    #Capping the DNI error at 100%
    if dataset_v2.iloc[i]["DNI_error"] > 1:
        dataset_v2.at[i, "DNI_error"] = 1

#Calculating bad DNI datapoints
badDNIdatapoints = len(dataset_v2[dataset_v2['DNI_error'] > 0.04])
ratioBadDNIpoints = badDNIdatapoints / rowNum

#Adding data to dataframe
dataset_v2["Total number of time stamps"] = ""
dataset_v2.at[0, 'Total number of time stamps'] = (rowNum + 1)

dataset_v2["Number of missing time stamps"] = ""
dataset_v2.at[0, 'Number of missing time stamps'] = missingTimestamps

dataset_v2["Number of DNI error data points above 4%"] = ""
dataset_v2.at[0, 'Number of DNI error data points above 4%'] = badDNIdatapoints

dataset_v2["Ratio of DNI error data points to the total number of data points"] = ""
dataset_v2.at[0, 'Ratio of DNI error data points to the total number of data points'] = ratioBadDNIpoints

#dataset_v2.drop(columns=["Total number of time stamps", 'Number of DNI error data points above 4%', "Percentage of the total DNI data points that are flagged as an error"], inplace=True)

#Exporting the dataset to the Excel Sheet and formatting
dataset_v2.to_excel(writer, sheet_name='Data_Quality_Check', index = False)
workbook = writer.book
worksheet = writer.sheets['Data_Quality_Check']

worksheet.freeze_panes(1, 0)

format_highlight = workbook.add_format({'bg_color':   '#f7e29f', 'font_color': '#141518'})

worksheet.conditional_format('G2:J2', {'type': 'cell',
                                       'criteria': '>=',
                                       'value':     0, 
                                       'format': format_highlight})

#more than 2 months of data
if (resolution == 1 and dataPointTotal > 1460) or (resolution == (60/3600) and dataPointTotal > 87600):
  resolution_Xaxis = 'months'
  format_Xaxis = 'mmm yy'

#less than 2 months of data
if (resolution == 1 and dataPointTotal <= 1460) or (resolution == (60/3600) and dataPointTotal <= 87600):
  resolution_Xaxis = 'days'
  format_Xaxis = 'dd/mm/yy'

#Plotting DNI Error figure 
chart = workbook.add_chart({'type': 'scatter'})

chart.add_series({
    'name':       ["Data_Quality_Check", 0, 5],
    'categories': ["Data_Quality_Check", 1, 0, len(dataset_v2.axes[0]), 0],
    'values':     ["Data_Quality_Check", 1, 5, len(dataset_v2.axes[0]), 5],
    'marker':     {'type': 'circle', 'size': 7, 'fill': {'color': '#ff8282'}, 'border': {'color': '#7c0000'}},
    })

chart.set_x_axis({
    'name': 'Date',
    'name_font': {'size': 12, 'bold': True},
    'num_format': format_Xaxis,
    'min': initialDate, 
    'max': lastDate,  
    })
chart.set_y_axis({
    'name': 'DNI Error',
    'major_gridlines': {'visible': False},
    'name_font': {'size': 12,'bold': True},
    'num_format': '0%',
    'min': 0, 'max': 1,    
    })

chart.set_title({'name': 'DNI error points between 10:00 and 15:00'})
chart.set_chartarea({'border': {'none': True}})
chart.set_legend({'none': True})

worksheet.insert_chart('H4', chart, {'x_scale': 2, 'y_scale': 2})

#Plotting DNI figure
chart = workbook.add_chart({'type': 'line'})

chart.add_series({
    'name':       ["Data_Quality_Check", 0, 3],
    'categories': ["Data_Quality_Check", 1, 0, len(dataset_v2.axes[0]), 0],
    'values':     ["Data_Quality_Check", 1, 3, len(dataset_v2.axes[0]), 3],
    'line':       {'color': '#8265a7'},
    })

chart.set_x_axis({
    'name': 'Date',
    'name_font': {'size': 12, 'bold': True},
    'num_format': format_Xaxis,
    'min': initialDate, 
    'max': lastDate,
    'major_unit': 1,
    'major_unit_type': resolution_Xaxis,
    'date_axis': True,
    })
chart.set_y_axis({
    'name': 'Solar irradiance (W/m^2)',
    'major_gridlines': {'visible': False},
    'name_font': {'size': 12,'bold': True},
    'num_format': '0',
    'min': 0,    
    })

chart.set_title({'name': 'Measured DNI'})
chart.set_chartarea({'border': {'none': True}})
chart.set_legend({'none': True})

worksheet.insert_chart('H34', chart, {'x_scale': 2, 'y_scale': 2})

#Plotting GHI and DHI on the same graph
chart = workbook.add_chart({'type': 'line'})
chart_DHI = workbook.add_chart({'type': 'line'})

chart.add_series({
    'name':       ["Data_Quality_Check", 0, 1],
    'categories': ["Data_Quality_Check", 1, 0, len(dataset_v2.axes[0]), 0],
    'values':     ["Data_Quality_Check", 1, 1, len(dataset_v2.axes[0]), 1],
    'line':       {'color': '#e27d5f'},
    })

chart_DHI.add_series({
    'name':       ["Data_Quality_Check", 0, 2],
    'categories': ["Data_Quality_Check", 1, 0, len(dataset_v2.axes[0]), 0],
    'values':     ["Data_Quality_Check", 1, 2, len(dataset_v2.axes[0]), 2],
    'line':       {'color': '#ffcb05'},
    })

chart.combine(chart_DHI)

chart.set_x_axis({
    'name': 'Date',
    'name_font': {'size': 12, 'bold': True},
    'num_format': format_Xaxis,
    'min': initialDate, 
    'max': lastDate,
    'major_unit': 1,
    'major_unit_type': resolution_Xaxis,
    'date_axis': True,
    })
chart.set_y_axis({
    'name': 'Solar irradiance (W/m^2)',
    'major_gridlines': {'visible': False},
    'name_font': {'size': 12,'bold': True},
    'num_format': '0',
    'min': 0,    
    })

chart.set_title({'name': 'Measured GHI and DHI'})
chart.set_chartarea({'border': {'none': True}})
chart.set_legend({'position': 'bottom'})

worksheet.insert_chart('H64', chart, {'x_scale': 2, 'y_scale': 2})

#Insert original data and save/download workbook
dataset_originalData.to_excel(writer, sheet_name="Original_Data", index= False)
writer.sheets['Original_Data'].freeze_panes(1, 0)
writer.save()

#Download new file
from google.colab import files
files.download(newPath)

**NOTE: If you would like to run this program again, click the play button of the second cell and upload another file.**